# Problem (Word2Vec as Matrix Factorization)

In this assignment you are supposed to apply SVD to training your own [word embedding model](https://en.wikipedia.org/wiki/Word_embedding) which maps English words to vectors of real numbers.

Skip-Gram Negative Sampling (SGNS) word embedding model, commonly known as **word2vec** ([Mikolov et al., 2013](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)), is usually optimized by stochastic gradient descent. However, the optimization of SGNS objective can be viewed as implicit matrix factorization objective as was shown in ([Levy and Goldberg, 2015](http://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization.pdf)).

###### 1. Notation

Assume we have a text corpus given as a sequence of words $\{w_1,w_2,\dots,w_n\}$ where $n$ may be larger than $10^{12}$ and $w_i \in \mathcal{V}$ belongs to a vocabulary of words $\mathcal{V}$. A word $c \in \mathcal{V}$ is called *a context* of word $w_i$ if they are found together in the text. More formally, given some measure $L$ of closeness between two words (typical choice is $L=2$), a word $c \in \mathcal{V}$ is called a context if $c \in \{w_{i-L}, \dots, w_{i-1}, w_{i+1}, \dots, w_{i+L} \}$ Let $\mathbf{w},\mathbf{c}\in\mathbb{R}^d$ be the *word embeddings* of word $w$ and context $c$, respectively. Assume they are specified by the mapping  $\Phi:\mathcal{V}\rightarrow\mathbb{R}^d$, so $\mathbf{w}=\Phi(w)$. The ultimate goal of SGNS word embedding model is to fit a good mapping $\Phi$.

Let $\mathcal{D}$ be a multiset of all word-contexts pairs observed in the corpus. In the SGNS model, the probability that word-context pair $(w,c)$ is observed in the corpus is modeled as the following distribution:
$$
P(\#(w,c)\neq 0|w,c) = \sigma(\mathbf{w}^\top \mathbf{c}) = \frac{1}{1 + \exp(-\mathbf{w}^\top \mathbf{c})},
$$
where $\#(w,c)$ is the number of times the pair $(w,c)$ appears in $\mathcal{D}$ and $\mathbf{w}^\top\mathbf{c}$ is the scalar product of vectors $\mathbf{w}$ and $\mathbf{c}$. Two important quantities which we will also use further are the number of times the word $w$ and the context $c$ appear in $\mathcal{D}$, which can be computed as
$$
\#(w) = \sum_{c\in\mathcal{V}} \#(w,c), \quad \#(c) = \sum_{w\in\mathcal{V}} \#(w,c).
$$

###### 2. Optimization objective

Vanilla word embedding models are trained by maximizing log-likelihood of observed word-context pairs, namely
$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d}.
$$

Skip-Gram Negative Sampling approach modifies the objective by additionally minimizing the log-likelihood of random word-context pairs, so called *negative samples*. This idea incorporates some useful linguistic information that some number ($k$, usually $k=5$) of word-context pairs *are not* found together in the corpus which usually results in word embeddings of higher quality. The resulting optimization problem is
$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) + k \cdot \mathbb{E}_{c'\sim P_\mathcal{D}} \log \sigma (-\mathbf{w}^\top\mathbf{c}) \right) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d},
$$
where $P_\mathcal{D}(c)=\frac{\#(c)}{|\mathcal{D}|}$ is a probability distribution over word contexts from which negative samples are drawn.

[Levy and Goldberg, 2015](http://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization.pdf) showed that this objective can be equivalently written as
$$
\mathcal{L} = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} f(w,c) = \sum_{w \in \mathcal{V}} \sum_{c \in \mathcal{V}} \left( \#(w,c) \log \sigma(\mathbf{w}^\top\mathbf{c}) + \frac{k\cdot\#(w)\cdot\#(c)}{|\mathcal{D}|} \log \sigma (-\mathbf{w}^\top\mathbf{c}) \right) \rightarrow \max_{\mathbf{w},\mathbf{c} \in \mathbb{R}^d},
$$
A crucial observation is that this loss function depends only on the scalar product $\mathbf{w}^\top\mathbf{c}$ but not on embedding $\mathbf{w}$ and $\mathbf{c}$ separately.

###### 3. Matrix factorization problem statement

Denote vocabulary size $|\mathcal{V}|$ as m. Let $W \in \mathbb{R}^{|\mathcal{V}|\times d}$ and $C \in \mathbb{R}^{|\mathcal{V}|\times d}$ be matrices, where each row $\mathbf{w}\in\mathbb{R}^d$ of matrix $W$ is the word embedding of the corresponding word $w$ and each row $\mathbf{c}\in\mathbb{R}^d$ of matrix $C$ is the context embedding of the corresponding context $c$. SGNS embeds both words and their contexts into a low-dimensional space $\mathbb{R}^d$, resulting in the word and context matrices $W$ and $C$. The rows of matrix $W$ are typically used in NLP tasks (such as computing word similarities) while $C$ is ignored. It is nonetheless instructive to consider the product $W^\top C = M$. Viewed this way, SGNS can be described as factorizing an implicit matrix M of dimensions $m \times m$ into two smaller matrices.

Which matrix is being factorized? A matrix entry $M_{wc}$ corresponds to the dot product $\mathbf{w}^\top\mathbf{c}$ . Thus, SGNS is factorizing a matrix in which each row corresponds to a word $w \in \mathcal{V}$ , each column corresponds to a context $c \in \mathcal{V}$, and each cell contains a quantity $f(w,c)$ reflecting the strength of association between that particular word-context pair. Such word-context association matrices are very common in the NLP and word-similarity literature. That said, the objective of SGNS does not explicitly state what this association metric is. What can we say about the association function $f(w,c)$? In other words, which matrix is SGNS factorizing?

### Task 1 (theoretical)

Solve SGNS optimization problem with respect to the $\mathbf{w}^\top\mathbf{c}$ and show that the matrix being factorized is
$$
M_{wc} = \mathbf{w}^\top\mathbf{c} = \log \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{k\cdot\#(w)\cdot\#(c)} \right)
$$
**Hint:** Denote $x=\mathbf{w}^\top\mathbf{c}$, rewrite SGNG optimization problem in terms of $x$ and solve it.

**Note:** This matrix is called Shifted Pointwise Mutual Information (SPMI) matrix, as its elements can be written as
$$
\text{SPMI}(w,c) = M_{wc} = \mathbf{w}^\top\mathbf{c} = \text{PMI}(w,c) - \log k
$$
and $\text{PMI}(w,c) = \log \left( \frac{\#(w,c) \cdot |\mathcal{D}|}{\#(w)\cdot\#(c)} \right)$ is the well-known pointwise mutual information of $(w,c)$.

**Proof:** now write your proof

### Task 2 (practical)

In [11]:
import os
import nltk
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

------

1. Download dataset [enwik8](http://mattmahoney.net/dc/enwik8.zip) of compressed Wikipedia articles and preprocess raw data with Perl script **main_.pl**. This script will clean all unnecessary symbols, make all words to lowercase, and produce only sentences with words.
```
wget http://mattmahoney.net/dc/enwik8.zip
unzip enwik8.zip
mkdir data
perl main_.pl enwik8 > data/enwik8.txt
```

In [12]:
import re
file = open("/workspace/data/enwik8.txt", "r")
doclist = [ line for line in file ]
docstr = '' . join(doclist)
sentences = re.split(r'[.!?]', docstr)
sentences = [sentence.split() for sentence in sentences if len(sentence) > 1]

In [13]:
len(sentences)

889156

In [14]:
# Load enwik 8
data = np.loadtxt("/workspace/data/enwik8.txt", dtype=str, delimiter='.')

# Create a list of sentences
sentences = [data[i].split() for i in data.nonzero()[0]]

In [15]:
print (sentences[1249])

['achilles', 'wrath', 'is', 'terrible', 'and', 'he', 'slays', 'many', 'trojan', 'warriors', 'and', 'allies', 'including', 'priam', 's', 'son', 'lycaon', 'whom', 'achilles', 'had', 'previously', 'captured', 'and', 'sold', 'into', 'slavery', 'but', 'who', 'had', 'been', 'returned', 'to', 'troy']


------

2. Construct the word vocabulary from the obtained sentences which enumerates words which occur more than $r=200$ times in the corpus.

In [16]:
def create_vocabulary(sentences, r=200):
    prevocabulary = {}
    for sentence in sentences:
        for word in sentence:
            if word in prevocabulary:
                prevocabulary[word] += 1
            else:
                prevocabulary[word] = 1

    vocabulary = {}
    idx = 0
    for word in prevocabulary:
        if (prevocabulary[word] > r):
            vocabulary[word] = idx
            idx += 1

    return vocabulary

In [17]:
vocab = create_vocabulary(sentences)

3. Scan the text corpus with sliding window of size $5$ and step $1$ (which corresponds to $L$=2) and construct co-occurrence word-context matrix $D$ with elements $D_{wc}=\#(w,c)$. Please, ignore words which occur less than $r=200$ times, but include them into the sliding window. Please, see the graphical illustration of the procedure described.

![Sliding window](sliding_window.png)

In [18]:
def create_corpus_matrix(sentences, vocabulary, window_size=5):
    """
    Create a co-occurrence matrix D from training corpus.
    """

    dim = len(vocabulary)
    D = np.zeros((dim, dim))
    s = window_size//2

    for sentence in sentences:
        l = len(sentence)
        for i in range(l):
            for j in range(max(0,i-s), min(i+s+1,l)):
                if (i != j and (sentence[i] in vocabulary) \
                    and (sentence[j] in vocabulary)):
                    c = vocabulary[sentence[j]]
                    w = vocabulary[sentence[i]]
                    D[c][w] += 1                  
    return D.T

In [24]:
len(sentences)

889156

In [23]:
D = create_corpus_matrix(sentences, vocab)

4. To find good word embeddings, [Levy and Goldberg, 2015](http://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization.pdf) proposed to find rank-$d$ SVD decomposition of Shifted Positive Pointwise Mutual Information (SPPMI) matrix
$$
U \Sigma V^\top \approx \text{SPPMI}(w,c) = \max\left(\text{SPMI}(w,c), 0 \right)
$$
and use $W=U\sqrt{\Sigma}$ as word embedding matrix. Your task is to reproduce their results. Write function constructs SPPMI matrix, computes its SVD and produces word-vectors matrix $W$.

In [43]:
def compute_sppmi(D, k):
    w_ = D.sum(axis=1)
    c_ = D.sum(axis=0)
    P = D.sum()
    w_v, c_v = np.meshgrid(w_, c_)
    B = k*(w_v*c_v)/float(P)
    #SPPMI = np.maximum(np.log(D)-np.log(B), 0)
    SPPMI = np.log(D)-np.log(B)
    return SPPMI

In [44]:
def compute_embeddings(D, k, d=200):
    # SVD initialization
    SPPMI = compute_sppmi(D, k)
    u, s, vt = svds(SPPMI, k=d)
    W_svd = u.dot(np.sqrt(np.diag(s))).T
    C_svd = np.sqrt(np.diag(s)).dot(vt)
    return W_svd

In [45]:
k = 5 # negative sampling parameter
W = compute_embeddings(D, k)

/Users/alexgrinch/miniconda3/envs/opensim-rl/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/Users/alexgrinch/miniconda3/envs/opensim-rl/lib/python3.6/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1817: RuntimeWarning: invalid value encountered in maximum
  eigvals = np.maximum(eigvals.real, 0)
/Users/alexgrinch/miniconda3/envs/opensim-rl/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/alexgrinch/miniconda3/envs/opensim-rl/lib/python3.6/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1827: RuntimeWarning: invalid value encountered in greater
  above_cutoff = (eigvals > cutoff)


5. Write class WordVectors using provided template

In [46]:
class WordVectors:
    
    def __init__(self, vocabulary, embedding_matrix):
        self.vocab = vocabulary
        self.W = embedding_matrix
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        
    def word_vector(self, word):
        """ Takes word and returns its word vector.
        """
        if word in self.vocab:
            vec = self.W[:,int(self.vocab[word])]
            vec = vec
        else:
            print ("No such word in vocabulary.")
            vec = None
            
        return vec
    
    def nearest_words(self, word, top=10, display=False):
        """ Takes word from the vocabulary and returns its top_n
        nearest neighbors in terms of cosine similarity.
        """

        vec = self.word_vector(word)[None, :]

        cosines = cosine_similarity(self.W.T, vec)[:, 0]
        args = np.argsort(cosines)[::-1]       

        nws = []
        for i in range(1, top+1):
            nws.append((self.inv_vocab[args[i]], round(cosines[args[i]], 3)))
            if (display):
                print (self.inv_vocab[args[i]], round(cosines[args[i]], 3))
        return nws

In [47]:
model = WordVectors(vocab, W)

In [48]:
model.nearest_words("anarchism")

[('dark', 0.0),
 ('weather', 0.0),
 ('stations', 0.0),
 ('college', 0.0),
 ('station', 0.0),
 ('c', 0.0),
 ('airport', 0.0),
 ('concentration', 0.0),
 ('trees', 0.0),
 ('less', 0.0)]

In [14]:
model.nearest_words("anarchism")

[('communism', 0.792),
 ('anarcho', 0.787),
 ('capitalism', 0.784),
 ('socialism', 0.752),
 ('liberalism', 0.727),
 ('criticisms', 0.705),
 ('capitalist', 0.662),
 ('fascism', 0.565),
 ('anarchist', 0.527),
 ('marxist', 0.518)]

In [38]:
model.nearest_words("ussr")

[('ukraine', 0.456),
 ('taiwan', 0.453),
 ('arabia', 0.448),
 ('expedition', 0.445),
 ('saudi', 0.444),
 ('revolt', 0.443),
 ('philippines', 0.437),
 ('syria', 0.411),
 ('conquest', 0.41),
 ('sweden', 0.406)]

In [15]:
model.nearest_words("ussr")

[('ukraine', 0.671),
 ('russia', 0.629),
 ('poland', 0.55),
 ('belarus', 0.538),
 ('yugoslavia', 0.538),
 ('romania', 0.517),
 ('serbia', 0.507),
 ('austria', 0.5),
 ('hungary', 0.466),
 ('bulgaria', 0.43)]

In [39]:
model.nearest_words("rap")

[('hip', 0.625),
 ('hop', 0.62),
 ('pop', 0.606),
 ('solo', 0.561),
 ('jazz', 0.556),
 ('funk', 0.545),
 ('albums', 0.519),
 ('singles', 0.519),
 ('punk', 0.506),
 ('blues', 0.498)]

In [16]:
model.nearest_words("rap")

[('hop', 0.828),
 ('hip', 0.817),
 ('funk', 0.758),
 ('rock', 0.737),
 ('punk', 0.706),
 ('music', 0.676),
 ('pop', 0.666),
 ('scene', 0.659),
 ('band', 0.657),
 ('jazz', 0.625)]

6. Calculate top 10 nearest neighbors with corresponding cosine similarities for the words {**numerical, linear, algebra**} and submit them into telegram bot.

In [40]:
model.nearest_words("numerical")

[('harmonic', 0.48),
 ('dynamic', 0.479),
 ('measurement', 0.471),
 ('precision', 0.468),
 ('computational', 0.455),
 ('derivative', 0.445),
 ('linear', 0.442),
 ('calculation', 0.44),
 ('construct', 0.43),
 ('determining', 0.43)]

In [17]:
model.nearest_words("numerical")

[('computation', 0.547),
 ('mathematical', 0.532),
 ('calculations', 0.499),
 ('polynomial', 0.485),
 ('calculation', 0.473),
 ('practical', 0.46),
 ('statistical', 0.456),
 ('symbolic', 0.455),
 ('geometric', 0.441),
 ('simplest', 0.438)]

In [41]:
model.nearest_words("linear")

[('finite', 0.525),
 ('binary', 0.499),
 ('integer', 0.497),
 ('discrete', 0.49),
 ('infinite', 0.484),
 ('differential', 0.479),
 ('geometric', 0.475),
 ('algorithms', 0.468),
 ('geometry', 0.465),
 ('algebra', 0.448)]

In [18]:
model.nearest_words("linear")

[('differential', 0.759),
 ('equations', 0.724),
 ('equation', 0.682),
 ('continuous', 0.674),
 ('multiplication', 0.674),
 ('integral', 0.672),
 ('algebraic', 0.667),
 ('vector', 0.654),
 ('algebra', 0.63),
 ('inverse', 0.622)]

In [42]:
model.nearest_words("algebra")

[('binary', 0.516),
 ('arithmetic', 0.508),
 ('theorem', 0.498),
 ('calculus', 0.489),
 ('geometry', 0.473),
 ('equation', 0.471),
 ('notation', 0.47),
 ('equations', 0.46),
 ('linear', 0.448),
 ('infinity', 0.427)]

In [19]:
model.nearest_words("algebra")

[('geometry', 0.795),
 ('calculus', 0.73),
 ('algebraic', 0.716),
 ('differential', 0.687),
 ('equations', 0.665),
 ('equation', 0.648),
 ('theorem', 0.647),
 ('topology', 0.634),
 ('linear', 0.63),
 ('integral', 0.618)]